# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
# from src.utils import prefilter_items

from src.recommenders import MainRecommender

In [3]:
data = pd.read_csv('../урок 2/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('../урок 2/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
item_features.shape

In [ ]:

popularity.shape

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [ ]:
# %%time

# model = AlternatingLeastSquares(factors=20, 
#                                 regularization=0.001,
#                                 iterations=15, 
#                                 calculate_training_loss=True, 
#                                 num_threads=4)

# model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
#           show_progress=True)

-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их, как делалось на вебинаре
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

In [41]:
def prefilter_items(data_train, item_feat):
    
    weeks_in_year = 52
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    data_train = data_train.loc[data_train['week_no'] >= data_train['week_no'].max() - weeks_in_year]
    
    popularity = data_train.groupby('item_id')[['quantity', 'sales_value']].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold', 'sales_value': 'price'}, inplace=True)
    popularity = popularity.loc[(popularity.n_sold > 0) & (popularity.price > 0)]
    popularity['new_sales_value'] = popularity.price / popularity.n_sold
    
    popularity = popularity.sort_values('new_sales_value', ascending=False).reset_index()
    
    # Уберем слишком дорогие товар
    
    popularity = popularity[3:]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    
    popularity = popularity.loc[popularity['new_sales_value'] > 1] 
    
    # Уберем самые непопулряные
    popularity = popularity.loc[popularity['n_sold'] > 10]
    
    # Уберем самые популярные (топ-3)
     
    popularity = popularity.sort_values('n_sold', ascending=False).reset_index()
    popularity = popularity[3:]
   
    # Уберем не интересные для рекоммендаций категории (department)
    small_count_departments = ['VIDEO RENTAL', 'AUTOMOTIVE', 'HOUSEWARES', 
            'PORK', 'POSTAL CENTER', 'GM MERCH EXP', 'CNTRL/STORE SUP', 
            'PROD-WHS SALES', 'DAIRY DELI', 'HBC', 'CHARITABLE', 'RX', 'TOYS',
            'PHOTO', 'DELI/SNACK BAR', 'GRO BAKERY', 'PHARMACY SUPPLY', 
            'ELECT &PLUMBING', 'MEAT-WHSE', 'VIDEO']
    
    
    item_feat_ids = item_feat.loc[~item_feat['department'].isin(small_count_departments), 'item_id']
    
    popularity = popularity.loc[popularity.item_id.isin(item_feat_ids), 'item_id'].tolist()
    
    top_items = popularity[:5000]
    
    # Добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_items), 'item_id'] = 999999 

    return data_train


def postfilter_items():
    pass

In [22]:
new_df = prefilter_items(data_train, item_features)
new_df

/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,999999,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,999999,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1189515,227,32310792652,362,999999,1,1.50,1089,-0.69,1611,52,0.0,0.0
1189516,227,32310792652,362,999999,1,1.79,1089,0.00,1611,52,0.0,0.0
1189517,1227,32330290028,356,999999,1,3.85,31863,-1.50,530,52,0.0,0.0
1189518,1227,32330290028,356,999999,1,5.49,31863,0.00,530,52,0.0,0.0


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,999999,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,999999,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1189515,227,32310792652,362,999999,1,1.50,1089,-0.69,1611,52,0.0,0.0
1189516,227,32310792652,362,999999,1,1.79,1089,0.00,1611,52,0.0,0.0
1189517,1227,32330290028,356,999999,1,3.85,31863,-1.50,530,52,0.0,0.0
1189518,1227,32330290028,356,999999,1,5.49,31863,0.00,530,52,0.0,0.0


In [23]:
new_df.item_id.nunique()

5726

5726

In [ ]:
def get_similar_items_recommendation(user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
    # your_code
    
    return res

def get_similar_users_recommendation(user, model, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    # your_code
    
    return res

In [4]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

from sklearn.metrics.pairwise import cosine_similarity

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from src.utils import prefilter_items


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, items_feat, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.items_features = items_feat
        
        self.user_item_matrix = self.prepare_matrix(data, self.items_features)  # pd.DataFrame
        
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = \
                self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data, items_feat):
        
        data = prefilter_items(data, items_feat)
        
        user_item_matrix = pd.pivot_table(data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', 
                                          aggfunc='count', 
                                          fill_value=0)
        
        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
    
    def get_recommendations(self, user, model, sparse_user_item, N=5):
        """Рекомендуем топ-N товаров"""
        res = [self.id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=self.userid_to_id[user], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        filter_items=[self.itemid_to_id[999999]],  # !!! 
                                        recalculate_user=True)]
        return res
    
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
        sparce_user_item = csr_matrix(self.user_item_matrix).tocsr()
        
        res = self.get_recommendations(user, own_recommender, sparce_user_item, N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        own_recommender = self.fit_own_recommender(self.user_item_matrix.T)
        
        sparce_user_item = csr_matrix(self.user_item_matrix).tocsr()
        
        res = self.get_recommendations(user, own_recommender, sparce_user_item, N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [5]:
main_rec = MainRecommender(data_train, item_features)

/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5726 [00:00<?, ?it/s]

In [6]:
main_rec.get_similar_items_recommendation(5, 10)

  0%|          | 0/5726 [00:00<?, ?it/s]

[975938,
 1073042,
 1004596,
 961353,
 1090587,
 935393,
 1022011,
 1080581,
 1063439,
 5995609]

In [ ]:
main_rec.get_similar_users_recommendation(5, 10)

  0%|          | 0/2497 [00:00<?, ?it/s]

----

Проверка, что все работает

In [ ]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

